In [2]:
import os
from tqdm import tqdm
import numpy as np
import tiktoken
from datasets import load_dataset # huggingface datasets

In [6]:
dataset = load_dataset("json", data_files='./data/json/dev.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/bm/.cache/huggingface/datasets/json/default-14dc56ae28f0b495/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# owt by default only contains the 'train' split, so create a test split
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=2357, shuffle=True)
split_dataset['val'] = split_dataset.pop('test') # rename the test split to val

In [9]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['sentences', 'labels', 'id'],
        num_rows: 65118
    })
    val: Dataset({
        features: ['sentences', 'labels', 'id'],
        num_rows: 7236
    })
})

In [16]:
from sentence_transformers import SentenceTransformer
from torch.nn.parallel import DistributedDataParallel as DDP
import torch
from torch.distributed import init_process_group, destroy_process_group
torch.set_default_device('mps')
init_process_group(backend='nccl')
model = DDP(SentenceTransformer('all-MiniLM-L6-v2'))

# we now want to tokenize the dataset. first define the encoding function (gpt2 bpe)
def process(example):
    embds = model.encode(example['sentences'])
    out = {'id': example['id'], 'labels': example['labels'], 'embeddings': embds}
    return out

# tokenize the dataset
tokenized = split_dataset.map(
    process,
    # remove_columns=['sentences'],
    desc="tokenizing the splits",
    num_proc=1,
)

destroy_process_group()

ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable RANK expected, but not set